In [ ]:
!pip install PyPDF2
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
import re
import os
from PyPDF2 import PdfReader
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import time

carpeta = 'PO'
#API sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
KEY = 'key.json'
SPREADSHEET_ID = '11IqC9lu7w8ggmdP4kjascyZb676TIbRWDYGan2AQPko'
creds = service_account.Credentials.from_service_account_file(KEY, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

def get_next_empty_row(sheet, spreadsheet_id, sheet_name, column_letter):
    range_ = f'{sheet_name}!{column_letter}:{column_letter}'
    result = sheet.values().get(
        spreadsheetId=spreadsheet_id,
        range=range_
    ).execute()
    
    values = result.get('values', [])
    return len(values) + 1

def append_values_to_spreadsheet(sheet, spreadsheet_id, range_, values, sheet_name):
    result = sheet.values().append(
        spreadsheetId=spreadsheet_id,
        range=f'{sheet_name}!{range_}',
        valueInputOption='USER_ENTERED',
        body={'values': values}
    ).execute()
    return result

def get_all_filenames(sheet, spreadsheet_id, sheet_name, column_letter):
    range_ = f'{sheet_name}!{column_letter}:{column_letter}'
    result = sheet.values().get(
        spreadsheetId=spreadsheet_id,
        range=range_
    ).execute()
    
    values = result.get('values', [])
    return [item[0] for item in values]


# Lista para almacenar los datos de todas las filas
data_to_append = []

all_filenames = get_all_filenames(sheet, SPREADSHEET_ID, 'PO', 'B')

for archivo in os.listdir(carpeta):
    if archivo.endswith('.pdf'):
        ruta_pdf = os.path.join(carpeta, archivo)
        
        with open(ruta_pdf, 'rb') as pdf_file:
            pdf_reader = PdfReader(pdf_file)
            
            # Obtener el nombre del archivo
            nombre_archivo = os.path.splitext(archivo)[0]
            if nombre_archivo in all_filenames:
                print(nombre_archivo, " ya se encuentra registrado")
            
            else:
                # Buscar el dato de 'delivery address'
                delivery_address = "None"
                for pagina_num in range(len(pdf_reader.pages)):
                    texto_pagina = pdf_reader.pages[pagina_num].extract_text()
                    if 'delivery address:' in texto_pagina.lower():
                        inicio = texto_pagina.lower().find('delivery address:') + len('delivery address:')
                        fin = texto_pagina.find('\n', inicio)
                        delivery_address = texto_pagina[inicio:fin].strip()
                        break
                # Buscar el dato de 'doc date'
                date = "None"
                for pagina_num in range(len(pdf_reader.pages)):
                    texto_pagina = pdf_reader.pages[pagina_num].extract_text()
                    if 'doc date:' in texto_pagina.lower():
                        inicio = texto_pagina.lower().find('doc date:') + len('doc date:')
                        fin = texto_pagina.find('\n', inicio)
                        date = texto_pagina[inicio:fin].strip()
                        break

                #'smp'
                codigo_smp = "None"
                for pagina_num in range(len(pdf_reader.pages)):
                    texto_pagina = pdf_reader.pages[pagina_num].extract_text()
                    if 'smp' in texto_pagina.lower():
                        inicio = texto_pagina.lower().find('smp')
                        fin = inicio + 14  # Longitud del código 'smp'
                        codigo_smp = texto_pagina[inicio:fin].strip()
                        break

                #'description'
                descripcion = "None"
                for pagina_num in range(len(pdf_reader.pages)):
                    texto_pagina = pdf_reader.pages[pagina_num].extract_text().lower()
                    pos_delivered = texto_pagina.find('total price currency')
                    if pos_delivered != -1:
                        inicio = pos_delivered + len('total price currency')+1
                        pos_nsn = texto_pagina.find('nsn', inicio)
                        if pos_nsn != -1:
                            fin = pos_nsn
                            descripcion = texto_pagina[inicio:fin].strip()
                            break

                # Buscar el dato de 'Total Value'
                price = "None"
                for pagina_num in range(len(pdf_reader.pages)):
                    texto_pagina = pdf_reader.pages[pagina_num].extract_text()
                    if 'total value:' in texto_pagina.lower():
                        inicio = texto_pagina.lower().find('total value:') + len('total value:')
                        fin = texto_pagina.find('\n', inicio)
                        price = texto_pagina[inicio:fin].strip()
                        break

                start = datetime.now()
                format_date_time = "%d/%m/%y"
                i = get_next_empty_row(sheet, SPREADSHEET_ID, 'PO', 'A')

                # Simplificar la estructura de la lista
                data_to_append.append([
                    str(start.strftime(format_date_time)),
                    nombre_archivo,
                    date,
                    delivery_address.replace(', .', '').strip(),
                    codigo_smp,
                    price.replace('COP',''),
                    descripcion
                ])

for i, data in enumerate(data_to_append, start=1):
    range_to_append = f'A{i}:G{i}'

    try:
        append_values_to_spreadsheet(
            sheet, SPREADSHEET_ID, range_to_append, [data], sheet_name='PO'
        )
    except Exception as e:
        print(f"Error en la operación de escritura para fila {i}: {e}")
print("Registro finalizado")

Registro finalizado
